# fine-tuning

In [1]:
# %%capture
# %pip install transformers trl

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

ModuleNotFoundError: No module named 'datasets'

In [3]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")

In [15]:
dataset = load_dataset("gofilipa/bedtime_stories")

Padding is necessary to account for different sizes of text in our dataset. 

*From the [docs on 🤗](https://huggingface.co/docs/transformers/en/pad_truncation): 
Batched inputs are often different lengths, so they can’t be converted to fixed-size tensors. Padding and truncation are strategies for dealing with this problem, to create rectangular tensors from batches of varying lengths. Padding adds a special padding token to ensure shorter sequences will have the same length as either the longest sequence in a batch or the maximum length accepted by the model. Truncation works in the other direction by truncating long sequences. In most cases, padding your batch to the length of the longest sequence and truncating to the maximum length a model can accept works pretty well.*

In [6]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [7]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs = 3, # how many times we iterate over the dataset as a whole
    learning_rate = 2e-4, # how many "steps" we take in adjusting the parameters to make up for loss
    weight_decay = 0.001, # way of regularizing the parameters
    report_to="none" # this is a new param, to avoid a login to W&B
)

In [20]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset['train'],
    dataset_text_field = "stories",
    tokenizer = tokenizer,
    args = training_params
)

<!--this cell is in markdown to avoid actually training a model when building the book-->
```python
trainer.train()
```

What's happening in the training process? Basically the process includes three functions:
- hypothesis function that makes a guess as to what word to generate following a given word
- loss function that calculates the difference between the guess and the actual word
- gradient descent that very slowly updates numbers as to minimize the loss

```python
trainer.model.save_pretrained("models")
trainer.tokenizer.save_pretrained("models")
```

```python
model = AutoModelForCausalLM.from_pretrained("./models")
tokenizer = AutoTokenizer.from_pretrained("./models")
```

```python
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=50)
```

```python
pipe("There once was a little girl named Filipa and")
```